# Installation and Imports

In [ ]:
pip install tensorflow matplotlib numpy pillow


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from PIL import Image
import os


# Loading the Dataset (Sample Images)

In [ ]:
def load_dataset(dataset_path, img_size=(64, 64)):
    images = []
    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(dataset_path, filename)
            img = Image.open(img_path)
            img = img.resize(img_size)
            img = np.array(img)
            images.append(img)
    images = np.array(images)
    images = images.astype(np.float32) / 127.5 - 1  # Normalize images to [-1,1]
    return images

dataset_path = 'path/to/your/dataset'  # Provide the path to your dataset
images = load_dataset(dataset_path)


# Model Preparation and Training

In [ ]:
# Generator
def build_generator():
    model = Sequential([
        layers.Dense(128, input_shape=(100,)),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(64 * 64 * 3, activation='tanh'),  # Output shape: (64, 64, 3)
        layers.Reshape((64, 64, 3))
    ])
    return model

# Discriminator
def build_discriminator():
    model = Sequential([
        layers.Flatten(input_shape=(64, 64, 3)),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(128),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Build GAN
def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    discriminator.trainable = False  # Freeze the discriminator for GAN training

    gan = Sequential([
        generator,
        discriminator
    ])
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)


# Training the GAN

In [ ]:
def train_gan(generator, discriminator, gan, images, epochs=1000, batch_size=32):
    num_batches = images.shape[0] // batch_size

    for epoch in range(epochs):
        for batch in range(num_batches):
            # Sample a random batch of real images
            idx = np.random.randint(0, images.shape[0], batch_size)
            real_images = images[idx]

            # Generate a batch of fake images
            noise = np.random.normal(0, 1, (batch_size, 100))
            fake_images = generator.predict(noise)

            # Train discriminator on real and fake images
            d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, 100))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Output loss and save generated images periodically
        if epoch % 100 == 0:
            print(f"Epoch: {epoch}, D Loss: {d_loss_real[0] + d_loss_fake[0]}, G Loss: {g_loss}")
            save_generated_images(generator, epoch)
            
def save_generated_images(generator, epoch, num_images=5):
    noise = np.random.normal(0, 1, (num_images, 100))
    generated_images = generator.predict(noise)

    # Visualize and save generated images
    for i in range(num_images):
        plt.imshow((generated_images[i] + 1) / 2)  # Convert back from [-1,1] to [0,1]
        plt.axis('off')
        plt.savefig(f'generated_images/epoch_{epoch}_image_{i}.png')
        plt.show()

train_gan(generator, discriminator, gan, images)


# Generating and Visualizing New Art

In [ ]:
def generate_and_visualize_new_art(generator, num_images=5):
    noise = np.random.normal(0, 1, (num_images, 100))
    generated_images = generator.predict(noise)

    # Visualize generated images
    for i in range(num_images):
        plt.imshow((generated_images[i] + 1) / 2)  # Convert back from [-1,1] to [0,1]
        plt.axis('off')
        plt.show()

generate_and_visualize_new_art(generator)


# Saving and Loading the Model

In [ ]:
# Save the models
generator.save('generator_model.h5')
discriminator.save('discriminator_model.h5')

# Load the models
loaded_generator = tf.keras.models.load_model('generator_model.h5')
loaded_discriminator = tf.keras.models.load_model('discriminator_model.h5')
